In [1]:
%gui qt6

# Qt6 stuff
from PyQt6.QtWidgets import QApplication
from PyQt6 import QtWidgets
from PyQt6 import QtGui
from PyQt6 import QtCore
from PyQt6.QtCore import pyqtSignal

# Plotting stuff
import pyqtgraph as pg
import matplotlib.pyplot as plt
import cmasher as cmr

# Science stuff
import numpy as np
import pandas as pd
from spectres import spectres

# Astropy stuff
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.units.quantity import Quantity
from astropy.io.ascii import read as ascii_read

# zHunter stuff
from zhunter import DIRS
from zhunter import io
from zhunter.misc import set_up_linked_vb
from zhunter.colors import get_gradient
from zhunter.spectroscopic_system import SpecSystemModel, SpecSystem
from zhunter.MainGraphicsWidget import MainGraphicsWidget
from zhunter.data_handler import DataHandler

# General stuff
import logging
import sys
from pathlib import Path
from itertools import cycle

In [2]:
log = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s [%(name)s] %(message)s')
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)

In [3]:
# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance

In [4]:
## Colors
from zhunter.colors import COLORS
color_style = 'kraken9'
colors = COLORS[color_style]

In [5]:
ergscm2AA = u.def_unit(
    s="ergscm2AA",
    represents=u.Unit("erg s^-1 cm^-2 AA^-1"),
    format={"latex": r"\mathrm{erg\,s^{-1}\,cm^{-2}\,\mathring{A}^{-1}}"},
)

## Generate fake data

In [6]:
# The fake data
N_data = 10000
N_spat = 40
wvlg = np.linspace(300, 900, N_data)
spat = np.linspace(-5, 5, N_spat)
flux = np.ones((wvlg.shape[0], spat.shape[0])) * np.exp(-(spat)**2/0.1)
flux += np.random.normal(0, 0.1, size=(wvlg.shape[0], spat.shape[0]))
flux = flux.T
unc = flux*0.1
wvlg_tell = wvlg
wvlg_sky_bkg = wvlg
tellurics = 1 - np.exp(-(wvlg-600)**2/(30)**2)
sky_bkg = np.exp(-(wvlg-800)**2/(10)**2)

wvlg *= u.nm
spat *= u.arcsec
flux *= ergscm2AA
unc *= ergscm2AA
data = DataHandler()
data.load_2D(wvlg, spat, flux, unc)


2023-03-21 11:10:47,366 DEBUG [zhunter.misc] Flux and uncertainty need to be rescaled (exponent: -1), rescaling them.


## Or real data

In [7]:
fname_2D = Path('/Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/XSHOOTER_esoreflex_2D.fits.gz')
fname_telluric = Path('/Users/palmerio/Code_projects/zHunter/src/zhunter/data/tellurics/sky_transimission_opt_to_nir.ecsv.gz')
fname_sky_bkg = Path('/Users/palmerio/Code_projects/zHunter/src/zhunter/data/sky_background/sky_background_norm_opt_to_nir.ecsv.gz')

wvlg, spat, flux, unc, header = io.read_fits_2D_spectrum(fname_2D)

spec_tell = io.read_generic_1D_spectrum(fname_telluric)
wvlg_tell = spec_tell.spectral_axis
tellurics = spec_tell.flux

spec_sky_bkg = io.read_generic_1D_spectrum(fname_sky_bkg)
wvlg_sky_bkg = spec_sky_bkg.spectral_axis
sky_bkg = spec_sky_bkg.flux

data = DataHandler()
data.load_2D(wvlg, spat, flux, unc)


2023-03-21 11:10:47,389 INFO [zhunter.io] Attempting to read file: /Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/XSHOOTER_esoreflex_2D.fits.gz
2023-03-21 11:10:48,172 WARNING [astropy] UnitsWarning: 'erg/s/cm2/Angstrom' contains multiple slashes, which is discouraged by the FITS standard
2023-03-21 11:10:48,186 DEBUG [zhunter.io] Using the FITS CD matrix.
2023-03-21 11:10:48,187 DEBUG [zhunter.io] PIX=1.0 VAL=-10.58000019073486 DELT=0.159999847412109
2023-03-21 11:10:48,188 DEBUG [zhunter.io] Using the FITS CD matrix.
2023-03-21 11:10:48,189 DEBUG [zhunter.io] PIX=1.0 VAL=533.66 DELT=0.0199999999999818
2023-03-21 11:10:48,192 INFO [zhunter.io] Attempting to read file: /Users/palmerio/Code_projects/zHunter/src/zhunter/data/tellurics/sky_transimission_opt_to_nir.ecsv.gz
2023-03-21 11:10:48,193 DEBUG [zhunter.io] ECSV file, reading with astropy ascii.read function


2023-03-21 11:10:51,068 DEBUG [zhunter.io] Found the following columns: ['awav(um)', 'flux_norm']
2023-03-21 11:10:51,068 DEBUG [zhunter.io] Did not find any name matching ['ERR', 'NOISE', 'SIGMA', 'UNC'] in ['awav(um)', 'flux_norm']
2023-03-21 11:10:51,069 DEBUG [zhunter.io] No unit could be parsed from column name 'flux_norm'
2023-03-21 11:10:51,070 INFO [zhunter.io] No units specified for flux, assuming ADU.
2023-03-21 11:10:51,210 WARNING [zhunter.io] No error/uncertainty found in file /Users/palmerio/Code_projects/zHunter/src/zhunter/data/tellurics/sky_transimission_opt_to_nir.ecsv.gz.
2023-03-21 11:10:51,212 INFO [zhunter.io] Attempting to read file: /Users/palmerio/Code_projects/zHunter/src/zhunter/data/sky_background/sky_background_norm_opt_to_nir.ecsv.gz
2023-03-21 11:10:51,213 DEBUG [zhunter.io] ECSV file, reading with astropy ascii.read function
2023-03-21 11:10:51,431 DEBUG [zhunter.io] Found the following columns: ['awav(um)', 'flux_norm']
2023-03-21 11:10:51,431 DEBUG [zh

## Create the Widgets that will hold the plots

In [8]:
win = MainGraphicsWidget(show=True)

In [9]:
win.set_up_plot(mode='2D', colors=colors)
win.ci.setBorder((100, 50, 100)) # this is to see where the Items' bounds are

2023-03-21 11:10:51,639 INFO [zhunter.MainGraphicsWidget] Setting up a new plot called '2D' in '2D' mode, with Region Of Interest 
2023-03-21 11:10:51,901 DEBUG [zhunter.MainGraphicsWidget] Setting up Region Of Interest
2023-03-21 11:10:51,904 DEBUG [zhunter.MainGraphicsWidget] Connecting signals and slots for ROI


### Add data to visualize

In [10]:
win.data = data
win.draw_data(data)

2023-03-21 11:10:51,910 INFO [zhunter.MainGraphicsWidget] Drawing 2D data
2023-03-21 11:10:51,966 WARNING [zhunter.MainGraphicsWidget] No extraction width specified, using 1 by default
2023-03-21 11:10:52,094 DEBUG [zhunter.MainGraphicsWidget] Drawing 1D data
2023-03-21 11:10:52,096 INFO [zhunter.MainGraphicsWidget] Extraction 2D spectrum from 533.6600 to 1020.0000 nm and from -3.080 to -2.080 arcsec
2023-03-21 11:10:52,194 DEBUG [zhunter.MainGraphicsWidget] Drawing 1D data


## Spectroscopic Systems

In [11]:
# Abstract Model to hold the list of spectroscopic systens
ssm = SpecSystemModel()


# QListView widget to view the model
specsysView = QtWidgets.QListView()
specsysView.setModel(ssm)
specsysView.show()

In [12]:
self = SpecSystem(6.318, PlotItem=win.ax1D, sys_type='abs',
                     color='cyan')
xmin=np.min(wvlg)
xmax=np.max(wvlg)
unit = None

if isinstance(xmin, Quantity):
    if unit is None:
        unit = xmin.unit
    else:
        xmin = xmin.to(unit)
        xmax = xmax.to(unit)
else:
    if unit is None:
        log.warning(
            "No units specified through xmin as Quantity"
            " or unit argument, using units of the line list."
        )
        unit = self.lines[self.wave_key].unit
        xmin = xmin * unit
        xmax = xmax * unit
    else:
        xmin = xmin * unit
        xmax = xmax * unit

self.plot_unit = unit

for w, n in zip(Quantity(self.lines[self.wave_key]), self.lines["name"]):
    if ("*" in n) and not self.show_fs:
        # If this is a fine structure line but show_fs is false, skip
        continue
    wave_obs = w * (1 + self.redshift)
    gt_min = (xmin is None) or (wave_obs >= xmin)
    lt_max = (xmax is None) or (wave_obs <= xmax)
    # If line observed wavelength is within the data range
    if gt_min and lt_max:
        if self.sys_type == "abs":
            line = pg.InfiniteLine(
                wave_obs.to(unit).value,
                span=(0.0, 0.8),
                pen=pg.mkPen(self.color, width=3),
                hoverPen=pg.mkPen(self.color, width=6),
                name=n,
                label=n,
                labelOpts={
                    "color": self.color,
                    "angle": 45,
                    "position": 1,
                },
                movable=True,
            )
        self.plotted_lines.append(line)


In [20]:
win.rect

PyQt6.QtCore.QRectF(533.66, -10.58000019073486, 486.33999999955745, 15.999984741210898)

In [13]:
self.pi.addItem(self.plotted_lines[0])

TypeError: 'QRectF' object is not callable

In [24]:
self.pi.vb.addItem(self.plotted_lines[1])

TypeError: 'QRectF' object is not callable

In [43]:
self.pi.vb.

In [37]:
self.pi.removeItem(xx)

In [38]:
xx = pg.InfiniteLine(
    600,
                span=(0.0, 0.8),
                pen=pg.mkPen(self.color, width=3),
                hoverPen=pg.mkPen(self.color, width=6),
#                 name='test_line',
                label='test_line',
#                 labelOpts={
#                     "color": self.color,
#                     "angle": 45,
#                     "position": 1,
#                 },
                movable=True,
            )
self.pi.addItem(xx)

In [14]:
# Spectroscopic system
specsys = SpecSystem(6.318, PlotItem=win.ax1D, sys_type='abs',
                     color='cyan')
specsys.draw(xmin=np.min(wvlg), xmax=np.max(wvlg))

ssm.specsystems.append((True, specsys))
specsys.edited.connect(ssm.layoutChanged.emit)
ssm.sort()
ssm.layoutChanged.emit()

2023-03-21 10:41:04,240 DEBUG [zhunter.spectroscopic_system] Drawing abs system at redshift : 6.31800


TypeError: 'QRectF' object is not callable

TypeError: 'QRectF' object is not callable

TypeError: 'QRectF' object is not callable

TypeError: 'QRectF' object is not callable

...

TypeError: 'QRectF' object is not callable

TypeError: 'QRectF' object is not callable

2023-03-21 10:43:39,659 DEBUG [zhunter.spectroscopic_system] Removed abs system at redshift 5.91137 from plot
2023-03-21 10:43:39,660 DEBUG [zhunter.spectroscopic_system] Drawing abs system at redshift : 5.91137
2023-03-21 10:43:39,696 INFO [zhunter.spectroscopic_system] Updated redshift of abs system to: 5.9114


Traceback (most recent call last):
  File "/Users/palmerio/opt/miniconda3/envs/zhunter_dev/lib/python3.9/site-packages/pyqtgraph/graphicsItems/GraphicsObject.py", line 36, in itemChange
    self.changeParent()
  File "/Users/palmerio/opt/miniconda3/envs/zhunter_dev/lib/python3.9/site-packages/pyqtgraph/graphicsItems/GraphicsItem.py", line 460, in changeParent
    self._updateView()
  File "/Users/palmerio/opt/miniconda3/envs/zhunter_dev/lib/python3.9/site-packages/pyqtgraph/graphicsItems/GraphicsItem.py", line 520, in _updateView
    self.viewTransformChanged()
  File "/Users/palmerio/opt/miniconda3/envs/zhunter_dev/lib/python3.9/site-packages/pyqtgraph/graphicsItems/InfiniteLine.py", line 609, in viewTransformChanged
    self.updatePosition()
  File "/Users/palmerio/opt/miniconda3/envs/zhunter_dev/lib/python3.9/site-packages/pyqtgraph/graphicsItems/InfiniteLine.py", line 546, in updatePosition
    vr = self.line.viewRect()
  File "/Users/palmerio/opt/miniconda3/envs/zhunter_dev/lib/py

2023-03-21 10:45:06,397 DEBUG [zhunter.MainGraphicsWidget] Key: Control, Mouse position: [4,383]
2023-03-21 10:45:06,398 DEBUG [zhunter.MainGraphicsWidget] key press didn't occur in any plot


In [16]:
index = specsysView.selectedIndexes()[0]
ssm.delete(index)
ssm.sort()

2023-03-21 10:46:26,552 DEBUG [zhunter.spectroscopic_system] Received request to delete system at redshift 5.91137
2023-03-21 10:46:26,558 DEBUG [zhunter.spectroscopic_system] Removed abs system at redshift 5.91137 from plot


## Velocity plot

In [14]:
from astropalmerio.spectra.conversions import *
from itertools import product

In [34]:
velLayout = pg.GraphicsLayoutWidget(show=True, border=(100,100,100))

qt.pointer.dispatch: skipping QEventPoint(id=0 ts=0 pos=0,0 scn=1243.24,694.147 gbl=1243.24,694.147 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1243.24,-694.147 last=-1243.24,-694.147 Δ 1243.24,694.147) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=1169.26,711.428 gbl=1169.26,711.428 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1169.26,-711.428 last=-1169.26,-711.428 Δ 1169.26,711.428) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=2 ts=0 pos=0,0 scn=1135.96,771.005 gbl=1135.96,771.005 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1135.96,-771.005 last=-1135.96,-771.005 Δ 1135.96,771.005) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=3 ts=0 pos=0,0 scn=1158.28,708.726 gbl=1158.28,708.726 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1158.28,-708.726 last=-1158.28,-708.726 Δ 1158.28,708.726) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=1211.88,233.44 gbl=1211.88,233.44 Released ellipse=(1x1 

In [53]:
velLayout.clear()

### Define lines to plot

In [29]:
lines = ascii_read(DIRS["DATA"] / "lines/basic_line_list.ecsv")
lnames = ['SiII_1260', 'OI_1302', 'SiII_1304']#, 'CII_1334', 'SiIV_1394']
z = 6.317
waves = np.array([l['wave'] for l in lines if l['name'] in lnames]) * lines['wave'].unit
waves_obs = waves.to('nm') * (1+z)

In [31]:
from zhunter.CheckBoxListWidget import CheckBoxListWidget
# Create checkboxlist widget
linelistView = CheckBoxListWidget()
linelistView.show()
linelistView.addItems(lines['name'])


In [46]:
lnames = []
for row in linelistView.getCheckedRows():
     lnames.append(linelistView.item(row).text())

lines_to_plot = {
            n: {"name": n, "rest_wave": rw, "obs_wave": ow}
            for n, rw, ow in zip(lnames, waves, waves_obs)
        }
lines_to_plot


{'SiII_1304': {'name': 'SiII_1304',
  'rest_wave': <Quantity 1302.1685 Angstrom>,
  'obs_wave': <Quantity 952.79669145 nm>},
 'SiII_1260': {'name': 'SiII_1260',
  'rest_wave': <Quantity 1304.3702 Angstrom>,
  'obs_wave': <Quantity 954.40767534 nm>}}

In [54]:
n_l = len(lines_to_plot)
if n_l <= 4:
    n_cols = 1
    n_rows = n_l
else:
    n_cols = int(n_l/2)
    n_rows = int(n_l/2)
    if n_l % 2 != 0:
        n_rows += 1

vel_min = -600 * u.Unit('km/s')
vel_max = 600 * u.Unit('km/s')
ref_lname = lnames[0]
for lines, indexes in zip(lines_to_plot.items(), product(range(1,n_cols+1), range(n_rows))):
    i, j = indexes
    lname, line = lines
    vel = wave_to_vel(wvlg_bins*u.nm, line['obs_wave'])
    imin = vel.searchsorted(vel_min)
    imax = vel.searchsorted(vel_max)
    vel_1D_spec = pg.PlotCurveItem(
        vel.value,
        flux_1D,
        pen=pg.mkPen(color=colors['spec']),
        stepMode='center')
    line['pi'] = velLayout.addPlot(title=lname.replace('_',' '), name=lname, col=i, row=j)
    line['pi'].addItem(vel_1D_spec)
    line['pi'].showGrid(x=True, y=True)
    line['pi'].hideButtons()
    line['pi'].setXLink(ref_lname)
    line['pi'].setXRange(-600, 600)
    line['pi'].setYRange(np.min(flux_1D[imin:imax]),
                         np.max(flux_1D[imin:imax]))
    line['pi'].vb.setLimits(
            xMin=np.min(vel.value), xMax=np.max(vel.value)
        )

velLayout.addLabel('Velocity (km/s)', col=1, row=n_rows+1, colspan=n_cols)
velLayout.addLabel('Flux', angle=-90, col=0, rowspan=n_rows)


In [52]:
line['pi'].vb.state

{'targetRange': [[-677.4596669241483, 677.4596669241483],
  [-7.016480923970568e-16, 1.4680171089385755e-15]],
 'viewRange': [[-677.4596669241483, 677.4596669241483],
  [-7.016480923970568e-16, 1.4680171089385755e-15]],
 'yInverted': False,
 'xInverted': False,
 'aspectLocked': False,
 'autoRange': [False, True],
 'autoPan': [False, False],
 'autoVisibleOnly': [False, False],
 'linkedViews': ['SiII_1304', None],
 'defaultPadding': 0.02,
 'mouseEnabled': [True, True],
 'mouseMode': 3,
 'enableMenu': True,
 'wheelScaleFactor': -0.125,
 'background': None,
 'logMode': [False, False],
 'limits': {'xLimits': [-132162.5789975226, 20603.443102673613],
  'yLimits': [-1e+307, 1e+307],
  'xRange': [None, None],
  'yRange': [None, None]}}

In [55]:
linelistView.clear()

In [3]:
from astropy.io.ascii import read as ascii_read


In [19]:
tab = ascii_read('/Users/palmerio/Code_projects/zHunter/src/zhunter/data/lines/basic_line_list.csv')
tab

name,wave
str10,float64
Ly_alpha,1215.6701
Ly_beta,1025.7223
Ly_gamma,972.5368
Ly_delta,949.7431
Ly_limit,911.7633
CII_1334,1334.5323
CII_1335*,1335.7077
CIV_1548,1548.2049
CIV_1550,1550.77845


In [26]:
tab['wave'].unit = u.Unit('Angstrom')

In [27]:
tab['wave'].unit

Unit("Angstrom")